# Battle of Neighbourhoods - Week 1

This notebook consist of two parts.

#### 1. Introduction
 1. Discussion of the business problem and the audience who would be interested in this project.

#### 2. Data Section
 2.  What data is used?
 2.  Importing Libraries
 2.  Credentials and Core location
 2.  Search for hotel & restaurant within 1 KM
 2.  Location of Hotels

        

## 1. Introduction/Business Problem:

### Discussion of the business problem and the audience who would be interested in this project.

#### Something about the tourist place - PUNE

I am from Pune,a beautiful tourist spot and also a education hub in India. Pune is the cultural capital of Maharashtra state. The most liveable city in India, Pune is one place in the country where you can easily feel the essence of composure. After all, it is surrounded by numerous beautiful hill stations. As it has a history dating back to early Medieval Period of India, it is home to several historical sites too. Therefore, it wouldn't be wrong to say that Pune is an ultimate city for a traveller who is looking forward to savouring the best of natural and historical sites. 

#### Opening of Hotel/Restaurant Shop

Coming down to business problem, I would like to open a hotel/restaurant in Pune. Pune is a utopia for food lovers. There are many competitiors in terms of hotel and restaurants. But keeping them in mind,i need to locate my hotel in place where more people are attracted and comfortable for a stay and a good meal.I want to bring foreign and local peoples attention towards my new hotel.I would like to flavour my retaurant recipe with Italian,American,typical south & north indian foods to grab their taste. So i would like to find the best possible place to open my hotel/restaurant.

###### The challenge is to find a suitable location for opening a new hotel / restaurant attracted to all local and foreign people in the centre of all famous venues.

#### Expected / Interested Audience:

Pune attracts lots of tourists every year . Some people stay for couple of days or more. Also they find some place for hangout or a good meal. Their main focus might be belonging to stay somewhere near to reach venues. As pune is educational hub there are lots of students so they search for some good places to hangout. So we may need to fascinate them all.

## 2.Data section

### 2.a What data is used?

We will be completely working on Foursquare data to explore and try to locate our new hotel where more venues like church, temples, beach, museums, memorials that are present nearby.

#### How will we be solving using this data?

We will looking for midpoint area of venues to locate our new hotel.Before that our major focus will be on all venues present in and around the core place of Pune.

Just a heads up on how many hotels are distributed now around Pune.We will perform some EDA on hotels & restaurants present in the tourist spot.On furthur notebook we will use Foursquare data to determine other venues as well.

### 2.b Importing Libraries

In [4]:
!pip install beautifulsoup4

In [5]:
!pip install geopy

In [6]:
# Import libraries
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("*********   Loaded library     ***********")

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

*********   Loaded library     ***********


### 2.c Credentials and Core location

In [ ]:
CLIENT_ID = 'YOPT3FBP41FRQSGIGL4O205JFQHH4Q4SXQ3DSQ3J1A3LNJXF' # your Foursquare ID
CLIENT_SECRET = 'ELKW2ZW5AVN54LZGMDPJOK2N2MGDOXAWWZQ1TKPPF2JTASDE' # your Foursquare Secret
VERSION = '20191028'
LIMIT = 150

address = "Pune, Maharashtra"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude =location.latitude   #8.079252 # location.latitude 
longitude =location.longitude #77.5499338 # location.longitude # 

pune='Pune location : {},{}'.format(latitude,longitude)
print(pune)

### 2.d Search for hotel & restaurant within 1 KM

In [ ]:
#Quering for hotel & restaurant

search_query = 'hotel'
search_query_res = 'restaurant'

radius = 1000
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)
#url

#### Send the GET Request of hotel & restaurants and examine the results

In [ ]:
results_hotel = requests.get(url_hotel).json()
results_restaurant = requests.get(url_restaurant).json()
#results_hotel

#### Get relevant part of JSON and transform it into a pandas dataframe

In [16]:
# assign relevant part of JSON to venues
venues_hotel = results_hotel['response']['venues']
venues_restaurant = results_restaurant['response']['venues']

# tranform venues into a dataframe and merging both data
dataframe_hotel = pd.json_normalize(venues_hotel)
dataframe_restaurant = pd.json_normalize(venues_restaurant)

dataframe = pd.concat([dataframe_hotel,dataframe_restaurant])

print("There are {} restaurants and hotels at Pune".format(dataframe.shape[0]))

There are 59 restaurants and hotels at Pune


#### Define information of interest and filter dataframe

In [17]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

  
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered
hotels_df=dataframe_filtered[['name','categories','distance','lat','lng','id']]
hotels_df.head()

,name,categories,distance,lat,lng,id
0,Hotel Gandharva,Indian Restaurant,582,18.522571,73.849066,4ed1190f490168f8d09f6d7c
1,Hotel Deccan Royal,Hotel,1006,18.519167,73.845223,4f124a07e4b00420555729cb
2,Hotel Courtyard Hinjewa Pune,None,229,18.520149,73.856164,4bdedb1dfe0e62b563000606
3,Hotel Kimaya Tapri,Smoke Shop,265,18.520430,73.856744,5101797ae4b0d12593642837
4,Hotel Silver Court,Seafood Restaurant,265,18.520430,73.856744,4fd4b657e4b02bec732a0728


#### 2.e Location of Hotels

In [19]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the pune

# add a red circle marker to represent the core location of kanyakumari
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Kanyakumari',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hotels_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(hotels_df.lat, hotels_df.lng, hotels_df.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
hotels_map

Hotel 'four seasons restaurant and bar' seems to be outlier.So this will be removed in next section.